[🥭 Entropy Markets](https://entropy.trade/) support is available at: [Docs](https://docs.entropy.trade/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/entropymarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

[![error](https://myerror.org/badge_logo.svg)](https://myerror.org/v2/gh/blockworks-foundation/entropy-explorer-examples/HEAD?labpath=PlaceAndCancelOrder.ipynb) [](https://myerror.org/v2/gh/blockworks-foundation/entropy-explorer-examples/HEAD?labpath=PlaceAndCancelOrder.ipynb) on error.

_🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

# 🥭 Refresh Orders

Solana's transaction mechanism allows for atomic cancel-and-replace of orders - either the entire transaction succeeds (old orders are cancelled, new orders are placed), or the entire transaction fails (no orders are cancelled, no orders are placed).

Neither Serum nor Entropy supports 'editing' or changing an order - to change the price or quantity for an order you must cancel it and replace it with an order with updated values.

This code will loop 3 times around:
* in one transaction: cancelling all perp orders and placing bid and ask perp orders on SOL-PERP
* wait for that transaction to confirm
* pause for 5 seconds

You can verify the transaction signatures in [Solana Explorer](https://explorer.solana.com/?cluster=mainnet) to see there is a single transaction containing a `CancelAllPerpOrders` instruction followed by two `PlacePerpOrder2` instructions. Since they're all in the same transaction, they will all succeed or all fail - if any instruction fails, the previous instructions are not committed to the chain, as if they never happened.

In [ ]:
import decimal
import entropy
import time

from solana.publickey import PublicKey

# Use our hard-coded mainnet wallet for DeekipCw5jz7UgQbtUbHQckTYGKXWaPQV4xY93DaiM6h.
# For real-world use you'd load the bytes from the environment or a file. Later we use
# its Entropy Account at AX7P7YXUNj7Ycp5VG5tweZqYngTk4kNFNZ7Zd2XHDrxB.
wallet = entropy.Wallet(bytes([181,213,227,47,41,229,109,138,15,82,26,7,230,184,88,102,197,215,238,155,136,196,138,92,98,154,67,68,47,140,90,40,248,149,223,193,241,51,4,196,126,32,211,66,90,137,249,160,132,246,38,29,88,16,252,116,12,83,117,158,40,98,178,54]))

with entropy.ContextBuilder.build(cluster_name="mainnet") as context:
    group = entropy.Group.load(context)
    account = entropy.Account.load(context, PublicKey("AX7P7YXUNj7Ycp5VG5tweZqYngTk4kNFNZ7Zd2XHDrxB"), group)
    market_operations = entropy.operations(context, wallet, account, "SOL-PERP", dry_run=False)
    market_instructions: entropy.PerpMarketInstructionBuilder = entropy.instruction_builder(context, wallet, account, "SOL-PERP", dry_run=False)

    signers: entropy.CombinableInstructions = entropy.CombinableInstructions.from_wallet(wallet)

    for counter in range(3):
        print("\n\nOrders:")
        orderbook = market_operations.load_orderbook()
        print(orderbook)

        instructions = signers
        instructions += market_instructions.build_cancel_all_orders_instructions()
        buy = entropy.Order.from_values(side=entropy.Side.BUY,
                                      price=orderbook.top_bid.price,
                                      quantity=decimal.Decimal(1),
                                      order_type=entropy.OrderType.POST_ONLY,
                                      client_id=counter+10)
        print(buy)
        instructions += market_instructions.build_place_order_instructions(buy)
        sell = entropy.Order.from_values(side=entropy.Side.SELL,
                                       price=orderbook.top_ask.price,
                                       quantity=decimal.Decimal(1),
                                       order_type=entropy.OrderType.POST_ONLY,
                                       client_id=counter+20)
        print(sell)
        instructions += market_instructions.build_place_order_instructions(sell)

        signatures = instructions.execute(context)

        print("Waiting for refresh order transaction to confirm...\n", signatures)
        entropy.WebSocketTransactionMonitor.wait_for_all(
                context.client.cluster_ws_url, signatures, commitment="processed"
            )

        print("Sleeping for 5 seconds...")
        time.sleep(5)

print("Cleaning up...")
instructions = signers
instructions += market_instructions.build_cancel_all_orders_instructions()

cleanup_signatures = instructions.execute(context)

print("Waiting for cleanup transaction to confirm...\n", cleanup_signatures)
entropy.WebSocketTransactionMonitor.wait_for_all(
        context.client.cluster_ws_url, cleanup_signatures, commitment="processed"
    )

print("Example complete.")